<a href="https://colab.research.google.com/github/YoushaaMurhij/HackerEarth_Challenges/blob/master/Gala_Image_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!unzip -q Gala.zip

In [2]:
import pandas as pd
import numpy as np
import keras
from keras.applications.vgg19 import VGG19
from keras.applications.mobilenet import MobileNet
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.applications.nasnet import NASNetMobile
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input, decode_predictions
from keras.layers import Dense, Activation, Flatten, merge,Input
from keras.models import Model, Sequential
from keras.utils import np_utils
from keras import backend as K
from keras.layers import *
from keras.callbacks import *
import os
import glob
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Using TensorFlow backend.


Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
train = pd.read_csv('dataset/train.csv')
test  = pd.read_csv('dataset/test.csv')

s = train['Class'].tolist()

from collections import Counter
print(Counter(s).keys()) # equals to list(set(words))
print(Counter(s).values()) # counts the elements' frequency
print(len(Counter(s).keys()))

tg_dict = {"Food":0, "misc": 1, "Attire": 2,"Decorationandsignage":3}
def label_encode(x):
    return tg_dict[x]

train['Class'] = train['Class'].apply(label_encode)

images = train['Image'].tolist()
classes = train['Class'].tolist()

features=[]
labels=[]
path = 'dataset/Train Images/'
for i in range(0,5983):
  if os.path.isfile(path+str(images[i])):
    pic = image.load_img(path+str(images[i]), target_size=(224, 224))
    #print(path+str(images[i]))
    x = image.img_to_array(pic)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    features.append(x)
    labels.append(classes[i])
  else:
    print(path+str(images[i]), 'not present')
   
npfeatures = np.array(features)
print(npfeatures.shape)
img_dt = np.rollaxis(npfeatures, 1, 0)
print(img_dt.shape)
X = img_dt[0]
print(X.shape)
labels = np.array(labels)
Y = np_utils.to_categorical(labels,4)
print(Y.shape)
def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator


dict_keys(['Food', 'misc', 'Attire', 'Decorationandsignage'])
dict_values([2278, 1271, 1691, 743])
4
(5983, 1, 224, 224, 3)
(1, 5983, 224, 224, 3)
(5983, 224, 224, 3)
(5983, 4)


In [0]:
#pip install efficientnet

In [7]:
#import efficientnet.tfkeras as efn
IMAGE_SIZE=[224,224]
#pretrained_model = MobileNet(weights='imagenet', include_top=False ,input_shape=[*IMAGE_SIZE, 3])
#pretrained_model = efn.EfficientNetB7(input_shape=(224, 224, 3),weights='imagenet',include_top=False)
#pretrained_model = MobileNetV2(input_shape=[*IMAGE_SIZE, 3], alpha=1.0, include_top=False, weights='imagenet',pooling=None, classes=4)
pretrained_model = NASNetMobile(input_shape=None, include_top=False, weights='imagenet')

pretrained_model.trainable = True # tramsfer learning    
model = Sequential([
        pretrained_model,
        GlobalAveragePooling2D(),
        Dense(220, activation='relu'),
        Dense(220, activation='relu'),
        Dense(4, activation='softmax')
    ])
es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto', baseline=None, restore_best_weights=False)
model.compile(optimizer = 'adamax', loss='categorical_crossentropy', metrics=[f1_m])    
model.load_weights("model_NASNet.h5")
model.fit(X, Y, batch_size=32, epochs=15, validation_split=.1,callbacks=[es])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 5384 samples, validate on 599 samples
Epoch 1/15
5384/5384 [==============================] - 138s 26ms/step - loss: 0.0881 - f1_m: 0.9763 - val_loss: 1.4543 - val_f1_m: 0.7653
Epoch 2/15
5384/5384 [==============================] - 67s 12ms/step - loss: 0.0202 - f1_m: 0.9895 - val_loss: 1.5593 - val_f1_m: 0.7784
Epoch 3/15
5384/5384 [==============================] - 67s 12ms/step - loss: 0.0194 - f1_m: 0.9920 - val_loss: 1.7874 - val_f1_m: 0.7835
Epoch 4/15
5384/5384 [==============================] - 67s 12ms/step - loss: 0.0218 - f1_m: 0.9902 - val_loss: 1.9662 - val_f1_m: 0.7224
Epoch 5/15
5384/5384 [==============================] - 68s 13ms/step - loss: 0.0122 - f1_m: 0.9940 - val_loss: 1.4890 - val_f1_m: 0.7865
Epoch 6/15
5384/5384 [==============================] - 68s 13ms/step - loss: 0.0206 - f1_m: 0.9918 - val_loss: 1.6210 - val_f1_m: 0.7836
Epoch 7/15
5384/5384 [======

In [0]:

model.fit(X, Y, batch_size=32, epochs=10, validation_split=.1,callbacks=[es])
model.fit(X, Y, batch_size=64, epochs=10, validation_split=.1,callbacks=[es])
model.fit(X, Y, batch_size=32, epochs=10, validation_split=.1,callbacks=[es])
model.fit(X, Y, batch_size=64, epochs=10, validation_split=.1,callbacks=[es])

model.save_weights("model_NASNet.h5")

Train on 5384 samples, validate on 599 samples
Epoch 1/10
5384/5384 [==============================] - 68s 13ms/step - loss: 0.0166 - f1_m: 0.9906 - val_loss: 1.8899 - val_f1_m: 0.7431
Epoch 2/10
5384/5384 [==============================] - 68s 13ms/step - loss: 0.0164 - f1_m: 0.9921 - val_loss: 1.7841 - val_f1_m: 0.7570
Epoch 3/10
5384/5384 [==============================] - 68s 13ms/step - loss: 0.0129 - f1_m: 0.9925 - val_loss: 1.8013 - val_f1_m: 0.7762
Epoch 4/10
5384/5384 [==============================] - 67s 13ms/step - loss: 0.0204 - f1_m: 0.9899 - val_loss: 2.5606 - val_f1_m: 0.7079
Epoch 5/10
5384/5384 [==============================] - 67s 12ms/step - loss: 0.0113 - f1_m: 0.9939 - val_loss: 2.1504 - val_f1_m: 0.7714
Epoch 6/10
5384/5384 [==============================] - 67s 12ms/step - loss: 0.0198 - f1_m: 0.9920 - val_loss: 2.3778 - val_f1_m: 0.7462
Epoch 7/10
5384/5384 [==============================] - 67s 12ms/step - loss: 0.0191 - f1_m: 0.9905 - val_loss: 3.2706 - val_

In [0]:
'''
image_input = Input(shape = (224,224,3))
model = MobileNet(input_tensor = image_input, weights = 'imagenet')
print(model.summary())
last_layer = model.get_layer('fc2').output
x = Dense(220,activation='relu')(last_layer)
x = Dense(220,activation='relu')(x)
out = Dense(4, activation='softmax')(x)
classifier = Model(image_input,out)
print(classifier.summary())
for Layer in classifier.layers[:-3]:
    Layer.trainable = False
es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto', baseline=None, restore_best_weights=False)
classifier.compile(optimizer = 'adamax', loss='categorical_crossentropy', metrics=[f1_m])    
classifier.fit(X, Y, batch_size=32, epochs=10, validation_split=.1,callbacks=[es])
'''

In [0]:
images_test = test['Image'].tolist()
test_features=[]
path_test = 'dataset/Test Images/'
for i in range(0,3219):
  if os.path.isfile(path_test+str(images_test[i])):
    pic = image.load_img(path_test+str(images_test[i]), target_size=(224, 224))
    #print(path+str(images[i]))
    x = image.img_to_array(pic)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    test_features.append(x)
  else:
    print(path_test+str(images[i]), 'not present')

In [0]:
test_features = np.array(test_features)
print(test_features.shape)
test_features = np.rollaxis(test_features, 1, 0)
print(test_features.shape)
X_test = test_features[0]
print(X_test.shape)

In [0]:
preds = model.predict(X_test)

In [0]:
predictions=[]
for i in preds:
    predictions.append(np.argmax(i))

In [0]:
test['Class'] = predictions

In [0]:
gt_dict = dict((v,k) for k,v in tg_dict.items())

def inverse_encode(x):
    return gt_dict[x]

test['Class'] = test['Class'].apply(inverse_encode)

In [0]:
test.head(4)

In [0]:
test.to_csv('Submission.csv',header=True,index = None)